In [2]:
import pandas as pd
import torch
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms
from sklearn.model_selection import KFold
from torch.utils.data import random_split
import matplotlib.pyplot as plt
import os
from dataset import MultitaskDataset

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
# Parameters
batch_size = 32
num_epochs = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_folds = 10  # Number of folds

# Root directory for dataset
root_dir = '../../data/BiteCount'
folds_dir = './folds/'  # Directory containing fold CSVs


In [5]:
# Load the corresponding training and validation CSV files for the current fold
train_annotation_file = os.path.join(folds_dir, f'train_fold_{1}.csv')
val_annotation_file = os.path.join(folds_dir, f'test_fold_{1}.csv')

# Create dataset instances for the current fold
train_dataset = MultitaskDataset(root_dir=root_dir, 
                                    annotation_file=train_annotation_file,
                                    rgb=True)

val_dataset = MultitaskDataset(root_dir=root_dir, 
                                annotation_file=val_annotation_file,
                                rgb=True)

# Create DataLoaders for the current fold
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


In [6]:
len(train_dataset)

3864

In [8]:
train_dataset.frame_paths

[(None,
  '../../data/BiteCount/salient_poses/salient1/4ff8a74c52c529eceb817a1d89e76a3ae053583c2abc491ecacbe560484b2e7e_2.mp4/0.jpg',
  'salient1/4ff8a74c52c529eceb817a1d89e76a3ae053583c2abc491ecacbe560484b2e7e_2.mp4/0.jpg',
  'salient1'),
 (None,
  '../../data/BiteCount/salient_poses/salient1/4ff8a74c52c529eceb817a1d89e76a3ae053583c2abc491ecacbe560484b2e7e_2.mp4/1.jpg',
  'salient1/4ff8a74c52c529eceb817a1d89e76a3ae053583c2abc491ecacbe560484b2e7e_2.mp4/1.jpg',
  'salient1'),
 (None,
  '../../data/BiteCount/salient_poses/salient1/4ff8a74c52c529eceb817a1d89e76a3ae053583c2abc491ecacbe560484b2e7e_2.mp4/10.jpg',
  'salient1/4ff8a74c52c529eceb817a1d89e76a3ae053583c2abc491ecacbe560484b2e7e_2.mp4/10.jpg',
  'salient1'),
 (None,
  '../../data/BiteCount/salient_poses/salient1/4ff8a74c52c529eceb817a1d89e76a3ae053583c2abc491ecacbe560484b2e7e_2.mp4/11.jpg',
  'salient1/4ff8a74c52c529eceb817a1d89e76a3ae053583c2abc491ecacbe560484b2e7e_2.mp4/11.jpg',
  'salient1'),
 (None,
  '../../data/BiteCount/sali

In [7]:
len(val_dataset)

403

In [10]:
val_dataset.frame_paths

[(None,
  '../../data/BiteCount/salient_poses/salient1/549a581a8eb4d9e66844c0409c97091b8dd6dd632607739d780cfd503bffbac0.mp4/0.jpg',
  'salient1/549a581a8eb4d9e66844c0409c97091b8dd6dd632607739d780cfd503bffbac0.mp4/0.jpg',
  'salient1'),
 (None,
  '../../data/BiteCount/salient_poses/salient1/549a581a8eb4d9e66844c0409c97091b8dd6dd632607739d780cfd503bffbac0.mp4/1.jpg',
  'salient1/549a581a8eb4d9e66844c0409c97091b8dd6dd632607739d780cfd503bffbac0.mp4/1.jpg',
  'salient1'),
 (None,
  '../../data/BiteCount/salient_poses/salient1/549a581a8eb4d9e66844c0409c97091b8dd6dd632607739d780cfd503bffbac0.mp4/10.jpg',
  'salient1/549a581a8eb4d9e66844c0409c97091b8dd6dd632607739d780cfd503bffbac0.mp4/10.jpg',
  'salient1'),
 (None,
  '../../data/BiteCount/salient_poses/salient1/549a581a8eb4d9e66844c0409c97091b8dd6dd632607739d780cfd503bffbac0.mp4/11.jpg',
  'salient1/549a581a8eb4d9e66844c0409c97091b8dd6dd632607739d780cfd503bffbac0.mp4/11.jpg',
  'salient1'),
 (None,
  '../../data/BiteCount/salient_poses/salien

In [3]:
# Define transformations for training set and validation set
train_transforms = transforms.Compose([
    transforms.Resize((224, 244)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


# Initialize model and optimizer
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 1)
model = model.to(device)

model.load_state_dict(torch.load('bce_loss.pth'))

/home/sadat/miniconda3/envs/cowbytes/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/sadat/miniconda3/envs/cowbytes/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/tmp/ipykernel_1399625/1950707710.py:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorc

<All keys matched successfully>

In [ ]:
import os
import cv2

def extract_frames_from_video(video_path, output_folder):
    """
    Extracts frames from a video file and saves them to the specified output folder.
    
    Args:
        video_path (str): Path to the video file.
        output_folder (str): Folder to save the extracted frames.
    """
    os.makedirs(output_folder, exist_ok=True)
    
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    
    count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Save the frame as a JPEG file
        frame_filename = os.path.join(output_folder, f"{os.path.basename(video_path).split('.')[0]}_frame_{count}.jpg")
        cv2.imwrite(frame_filename, frame)
        count += 1
    
    cap.release()


In [ ]:
# Assuming 'dataset' is your entire dataset containing all frames
# Create a DataLoader for the dataset
prediction_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, collate_fn=custom_collate)

# Prepare lists to store results
frame_indices = []
predictions = []

# Set model to evaluation mode
model.eval()

# Disable gradient calculation for inference
with torch.no_grad():
    for batch_idx, (inputs, _) in enumerate(prediction_loader):
        inputs = inputs.to(device)
        
        # Forward pass to get logits
        logits = model(inputs)
        
        # Apply sigmoid to get probabilities
        probs = torch.sigmoid(logits)
        
        # Store frame indices and predictions
        frame_indices.extend(batch_idx * batch_size + i for i in range(len(probs)))
        predictions.extend(probs.cpu().numpy())

# Create a DataFrame with frame indices and predictions
results_df = pd.DataFrame({
    'frame': frame_indices,
    'prediction': predictions
})

# Save the DataFrame to a CSV file if needed
results_df.to_csv('predictions.csv', index=False)

print(results_df.head())  # Print the first few rows of the results DataFrame